In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import math
import matplotlib.dates as mdates
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.impute import SimpleImputer



# Models
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
# VIF
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


In [ ]:
df = pd.read_csv('/content/crop_data_cleaned.csv') # Import csv first
df2 = df.copy()

# Convert 'Area planted(ha)' and 'Area harvested(ha)' to numeric, coercing errors to NaN
df['Area planted(ha)'] = pd.to_numeric(df['Area planted(ha)'], errors='coerce')
df['Area harvested(ha)'] = pd.to_numeric(df['Area harvested(ha)'], errors='coerce')

FileNotFoundError: [Errno 2] No such file or directory: '/content/crop_data_cleaned.csv'

In [ ]:
import pandas as pd

# --- 1️⃣ Load the dataset ---
df = pd.read_csv("crop_data_cleaned.csv")

# --- 2️⃣ Convert 'Production(mt)' to numeric ---
df["Production(mt)"] = pd.to_numeric(df["Production(mt)"], errors="coerce")

# --- 3️⃣ 🧠 Define your filters ---
crop = "BROCCOLI"        # Example: "CABBAGE"
municipality = "LA TRINIDAD"   # Example: "ATOK"
year = 2022             # Example: 2016

# --- 4️⃣ Filter dataset ---
filtered = df[
    (df["CROP"] == crop) &
    (df["MUNICIPALITY"] == municipality) &
    (df["YEAR"] == year)
]

# --- 5️⃣ Ensure month names are ordered correctly ---
month_order = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN",
               "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]

filtered["MONTH"] = filtered["MONTH"].str.upper().str.strip()
filtered["MONTH"] = pd.Categorical(filtered["MONTH"], categories=month_order, ordered=True)

# --- 6️⃣ Group by MONTH and sum Production ---
monthly_trend = (
    filtered.groupby("MONTH", observed=True)["Production(mt)"]
    .sum()
    .reset_index()
    .sort_values("MONTH")
)

# --- 7️⃣ Convert to kilograms ---
monthly_trend["Production(kg)"] = monthly_trend["Production(mt)"] * 1000

# --- 8️⃣ Print text-based results ---
print(f"📊 Monthly Production Trend for {crop} in {municipality} ({year})\n")
for _, row in monthly_trend.iterrows():
    print(f"{row['MONTH']}: {row['Production(mt)']:.2f} mt ({row['Production(kg)']:,.0f} kg)")

# --- 9️⃣ Optional: total for the year ---
total_mt = monthly_trend["Production(mt)"].sum()
print(f"\nTOTAL: {total_mt:.2f} mt ({total_mt * 1000:,.0f} kg)")


In [ ]:
df['Productivity(mt/ha)'] = df['Productivity(mt/ha)'].replace([r'^\s*$', '#REF!'], np.nan, regex=True)
df['Productivity(mt/ha)'] = df['Productivity(mt/ha)'].astype(float)
df = df.drop('Area planted(ha)', axis=1)
df3 = df.copy()

In [ ]:
# Descriptive Statistics
df.drop(['YEAR'], axis=1).describe()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax = sns.heatmap(df.corr(numeric_only=True), cmap='YlGnBu', annot=True)
ax.set_title('Feature Correlation', size=15)
ax.tick_params(axis='x', rotation=45)
plt.show()

In [ ]:
# VIF
vif_dataframe = df3.copy()
vif_dataframe = vif_dataframe.select_dtypes(include=['number'])
vif_dataframe = vif_dataframe.drop('Productivity(mt/ha)', axis=1)
vif_dataframe = vif_dataframe.replace([np.inf, -np.inf], np.nan).dropna()

constant = sm.add_constant(vif_dataframe)

vif_data = pd.DataFrame()
vif_data['Variable'] = constant.columns
vif_data['VIF'] = [variance_inflation_factor(constant.values, i) for i in range(constant.shape[1])]
vif_data = vif_data[vif_data['Variable'] != 'const'].reset_index(drop=True)
print(vif_data)

In [ ]:
# Select numerical predictors (excluding the target variable 'Production (mt)')
predictors = df[[ 'Area harvested(ha)', 'Productivity(mt/ha)']]

# Drop rows with missing values in predictors
predictors = predictors.dropna()

# Add constant term for VIF calculation
X = add_constant(predictors)

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display the VIF results
print(vif_data)

In [ ]:
def preprocess(df):
    # Convert 'Production(mt)' to numeric, coercing errors to NaN
    df['Production(mt)'] = pd.to_numeric(df['Production(mt)'], errors='coerce')

    # Drop rows where 'Production(mt)' is NaN
    df = df.dropna(subset=['Production(mt)'])

    # Preprocess data (drop columns, fill NaNs with 0)
    df = df.drop('Productivity(mt/ha)', axis=1)
    df = df.drop(['YEAR'], axis = 1)

    # Feature and Target (X for features and y for target)
    X = df[['MUNICIPALITY', 'FARM TYPE', 'MONTH', 'CROP', 'Area harvested(ha)']]
    y = df['Production(mt)']

    # Explicitly convert numerical column to float
    X['Area harvested(ha)'] = pd.to_numeric(X['Area harvested(ha)'], errors='coerce')

    # Encode categorical features (turn unique values into columns)
    onehotencoder = OneHotEncoder()
    encoded = onehotencoder.fit_transform(X[categorical_cols]).todense() # Use categorical_cols
    names = onehotencoder.get_feature_names_out(categorical_cols)
    X_encoded_df = pd.DataFrame(encoded, index=X.index) # Use the index from X
    X_encoded_df.columns = names

    # Rename columns
    for col in X_encoded_df.columns:
        X_encoded_df.rename({col: col.removeprefix('MUNICIPALITY_')}, axis = 1, inplace=True)
        X_encoded_df.rename({col: col.removeprefix('FARM TYPE_')}, axis = 1, inplace=True)
        X_encoded_df.rename({col: col.removeprefix('MONTH_')}, axis = 1, inplace=True)
        X_encoded_df.rename({col: col.removeprefix('CROP_')}, axis = 1, inplace=True)


    # Concat features to main dataframe
    add_to_X = X[numeric_cols] # Use numerical_cols
    X_frame = pd.concat([X_encoded_df, add_to_X], axis = 1)

    # Ensure all columns in X_frame are numeric
    X_frame[numeric_cols] = X_frame[numeric_cols].astype(float)


    # Convert dataframe to array
    X = X_frame
    y = y

    return df, X, y, X_frame, X_frame.index

# Define categorical and numerical columns globally for use in preprocess and ColumnTransformer
categorical_cols = ['MUNICIPALITY', 'FARM TYPE', 'MONTH', 'CROP']
numeric_cols = ['Area harvested(ha)']

# Unpack datrame, X for features and y for target
df, X, y, X_frame, cleaned_index = preprocess(df3)

In [ ]:
df = pd.read_csv('/content/Benguet Commodity_2024.csv') # Import csv first
df2 = df.copy()

# Print the first few rows to inspect the columns
print("First 5 rows of the DataFrame after loading:")
display(df.head())

In [ ]:
# Example of OneHoEencoder on MUNICIPALITY
encoder = OneHotEncoder()

encoded_sample = df[['MUNICIPALITY']].drop_duplicates()
encoded_array = encoder.fit_transform(encoded_sample).todense()
encoded_feature_names = encoder.get_feature_names_out(['MUNICIPALITY'])
encoded_df = pd.DataFrame(encoded_array, columns=encoded_feature_names)
combined_df = pd.concat([encoded_sample.reset_index(drop=True), encoded_df], axis=1)
combined_df

In [ ]:
# Features
X

In [ ]:
X_frame

In [ ]:
# Target
y

In [ ]:
# Convert 'Area planted(ha)' and 'Area harvested(ha)' to numeric, coercing errors to NaN
df['Area planted(ha)'] = pd.to_numeric(df['Area planted(ha)'], errors='coerce')
df['Area harvested(ha)'] = pd.to_numeric(df['Area harvested(ha)'], errors='coerce')

# Drop rows where the target variable 'Production(mt)' is NaN
df.dropna(subset=['Production(mt)'], inplace=True)

X = df.drop(columns=['Production(mt)'])   # All columns except the target
y = df['Production(mt)']                  # Target column
cleaned_index = df.index                  # Keep aligned index

print("Length of X:", len(X))
print("Length of y:", len(y))
print("Length of cleaned_index:", len(cleaned_index))

X_train, X_test, y_train, y_test, train_index, test_index = train_test_split(
    X, y, cleaned_index, test_size=0.2, random_state=0
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# Define a custom transformer to convert sparse output to dense
class ToDense(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # Convert to dense if it's a sparse matrix, otherwise return as is
        if hasattr(X, 'toarray'):
            return X.toarray()
        return X


# Identify categorical and numeric columns - Explicitly list them based on the structure of X_train/X_test DataFrames
categorical_cols = ['MUNICIPALITY', 'FARM TYPE', 'MONTH', 'CROP']
numeric_cols = ['Area harvested(ha)', 'Area planted(ha)', 'YEAR']  # Removed 'Productivity(mt/ha)' from numeric_cols

print("Categorical Columns:", list(categorical_cols))
print("Numeric Columns:", list(numeric_cols))

# Define Preprocessor (handles encoding + scaling + imputation)
preprocessor = ColumnTransformer(
    transformers=[
        # Make OneHotEncoder output dense directly
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols),
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numeric_cols)
    ],
    remainder='passthrough'  # Keep other columns if any (though none expected in this case)
)

# Random Forest pipeline only
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=0))
])

# Define grid for Random Forest
rf_grid = {
    'model__max_depth': [100, 200, 300],
    'model__n_estimators': [100, 200],
    'model__min_samples_split': [2, 5]
}

# Train-test split
df_cleaned_target = df2.copy()
df_cleaned_target['Production(mt)'] = pd.to_numeric(df_cleaned_target['Production(mt)'], errors='coerce')
df_cleaned_target.dropna(subset=['Production(mt)'], inplace=True)

X = df_cleaned_target.drop(columns=['Production(mt)'])
y = df_cleaned_target['Production(mt)']
cleaned_index = df_cleaned_target.index

X_train, X_test, y_train, y_test, train_index, test_index = train_test_split(
    X, y, cleaned_index, test_size=0.2, random_state=0
)

# Convert numeric columns to float
for col in numeric_cols:
    if col in X_train.columns:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    if col in X_test.columns:
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce')

# Drop rows with NaN in numeric columns
train_data_cleaned = pd.concat([X_train, y_train], axis=1).dropna(subset=numeric_cols + [y_train.name])
X_train = train_data_cleaned.drop(y_train.name, axis=1)
y_train = train_data_cleaned[y_train.name]

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Train model using GridSearchCV
print("\nData types of X_train before fitting:")
print(X_train.dtypes)

print("\nChecking for NaN in X_train numerical columns before preprocessing:")
for col in numeric_cols:
    if col in X_train.columns and X_train[col].isnull().any():
        print(f"Column '{col}' contains NaN values.")

# Apply preprocessing to check for NaNs after
X_train_processed = preprocessor.fit_transform(X_train)

print("\nChecking for NaN in X_train after preprocessing:")
if np.isnan(X_train_processed).any():
    print("NaN values found in X_train after preprocessing.")
else:
    print("No NaN values found in X_train after preprocessing.")

# Train only Random Forest
print("\nTraining the Random Forest model...")
rf_model = GridSearchCV(rf_pipeline, rf_grid, n_jobs=-1, cv=5)
rf_model.fit(X_train, y_train)
print(f"Best Parameters for Random Forest: {rf_model.best_params_}")

print("\nRandom Forest model trained successfully!")


In [ ]:
pd.DataFrame(rf_model.cv_results_)

In [ ]:
rf_model.cv_results_

In [ ]:
# Extract the trained pipeline
best_model = rf_model.best_estimator_

# Get the preprocessor (ColumnTransformer)
preprocessor = best_model.named_steps['preprocessor']

# Get the transformed feature names (after encoding/scaling)
feature_names = preprocessor.get_feature_names_out()

# Get the Random Forest feature importances
importances = best_model.named_steps['model'].feature_importances_

# Build the DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display results
print(importance_df)


In [ ]:
# Record the metrics for the Random Forest model
y_pred = rf_model.predict(X_test)

# Compute metrics
accuracy = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)


In [ ]:
metrics = pd.DataFrame({
    'Model': ['Random Forest'],
    'R2 Score': [accuracy],
    'Mean Absolute Error': [mae],
    'Mean Squared Error': [mse],
    'Root Mean Squared Error': [rmse]
})

metrics


In [ ]:
columns = ['Area planted(ha)','Area harvested(ha)']

def plot_distribution(columns):
    fig, axes = plt.subplots(1, len(columns), figsize=(6 * len(columns), 5))

    if len(columns) == 1:
        axes = [axes]

    for i, column in enumerate(columns):
        # Convert column to numeric, coercing errors to NaN, then drop NaNs for plotting
        plot_data = pd.to_numeric(df2[column], errors='coerce').dropna()
        sns.kdeplot(plot_data, fill=True, ax=axes[i], color='skyblue')
        axes[i].set_title(f'Distribution of {column}', fontsize=12)
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Density')

    plt.tight_layout()
    plt.show()

plot_distribution(columns)

In [ ]:
# Get true and predicted values for Random Forest
y_true = y_test
y_pred = rf_model.predict(X_test)

# Create scatter plot
plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_true, y=y_pred, color='skyblue', alpha=0.7, edgecolor='k')

# Diagonal line (perfect prediction)
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], color='red', linestyle='--')

# Label setup
label_x = (y_true.min() + y_true.max()) / 2
label_y = y_pred.max()

# Add a label for outliers
plt.text(label_x, label_y, "Big Outlier(s)", fontsize=10, fontweight='bold', color='black')

# Highlight points with large prediction errors (>1500 difference)
for actual, predicted in zip(y_true, y_pred):
    if abs(predicted - actual) > 1500:
        plt.annotate(
            '', xy=(actual, predicted), xytext=(label_x, label_y),
            arrowprops=dict(facecolor='black', arrowstyle='->', linewidth=0.8)
        )

plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Random Forest: Prediction vs Actual (Outlier Detection)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
"""
This plot shows the overall trend of harvested area (in hectares) over the years across all municipalities combined.

- The x-axis represents the years, showing temporal progression.
- The y-axis indicates the total area harvested.
- The line plot highlights how the harvested area has increased, decreased, or remained stable over time.
- This visualization provides a clear view of the general change in agricultural activity on a yearly basis.
"""

# Yearly Crop Production Trend
data = df2[['MUNICIPALITY', 'YEAR', 'Area harvested(ha)']]

# Set minimal theme
sns.set_theme(style="darkgrid")

# Plot
plt.figure(figsize=(10, 5))
sns.lineplot(data=data, x='YEAR', y='Area harvested(ha)', marker='o', color='royalblue', errorbar=None)

# Title and labels
plt.title("Harvested Over Years", fontsize=14)
plt.xlabel("Year")
plt.ylabel("Area Harvested (ha)")

plt.tight_layout()
plt.show()



In [ ]:
"""
This figure illustrates the yearly trends in harvested area (in hectares) for each municipality included in the dataset.
Each subplot represents a specific municipality, showing how the harvested area has changed over time.
The x-axis indicates the years, allowing for the observation of long-term trends in crop production,
while the y-axis is shared across all subplots to maintain a consistent scale and facilitate direct comparison between municipalities.

The visualization reveals that municipalities such as Bakun, Buguias, Kabayan, Kibungan, and Mankayan produce significantly larger amounts of crops compared to other areas,
with Buguias emerging as the highest producer. Additionally, the plots indicate a general upward trend in harvested areas across all municipalities,
suggesting a steady increase in agricultural production over the years.

"""

municipalities = df2['MUNICIPALITY'].unique()
n = len(municipalities)
cols = 3
rows = math.ceil(n / cols)

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), sharey=True)
axes = axes.flatten()

# Plot each municipality
for i, municipality in enumerate(municipalities):
    data = df2[df2['MUNICIPALITY'] == municipality]
    sns.lineplot(ax=axes[i], data=data, x='YEAR', y='Area harvested(ha)', marker='o', color='royalblue', errorbar=None)
    axes[i].set_title(municipality, fontsize=10)
    axes[i].set_xlabel("YEAR")
    axes[i].tick_params(axis='x', labelrotation=45, labelsize=8)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Yearly Harvest Trend Across Municipalities', fontsize=16)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
municipalities = df2['MUNICIPALITY'].unique()
n = len(municipalities)

# 2 columns: one for 'Area harvested', one for 'Production'
cols = 2
rows = n

fig, axes = plt.subplots(rows, cols, figsize=(cols * 7, rows * 3), sharey=False)
if rows == 1:
    axes = axes.reshape(1, 2)

for i, municipality in enumerate(municipalities):
    data = df2[df2['MUNICIPALITY'] == municipality]

    # Area harvested plot (left column)
    sns.lineplot(ax=axes[i, 0], data=data, x='YEAR', y='Area harvested(ha)', marker='o', color='royalblue', errorbar=None)
    axes[i, 0].set_title(f"{municipality} - Area Harvested", fontsize=10)
    axes[i, 0].set_xlabel("")
    axes[i, 0].set_ylabel("Area (ha)")
    axes[i, 0].tick_params(axis='x', labelrotation=45, labelsize=8)

    # Production plot (right column)
    sns.lineplot(ax=axes[i, 1], data=data, x='YEAR', y='Production(mt)', marker='s', color='darkgreen', errorbar=None)
    axes[i, 1].set_title(f"{municipality} - Production", fontsize=10)
    axes[i, 1].set_xlabel("")
    axes[i, 1].set_ylabel("Production (mt)")
    axes[i, 1].tick_params(axis='x', labelrotation=45, labelsize=8)

# Adjust layout
fig.suptitle('Yearly Trends in Area Harvested and Production per Municipality', fontsize=16)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# Group by MONTH and CROP, and get mean Production
# Convert 'Production(mt)' to numeric, coercing errors to NaN
df2['Production(mt)'] = pd.to_numeric(df2['Production(mt)'], errors='coerce')

avg_df = df2.groupby(['MONTH', 'CROP'], as_index=False)['Production(mt)'].mean()
month_order = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# Convert 'MONTH' to ordered categorical for sorting on x-axis
avg_df['MONTH'] = pd.Categorical(avg_df['MONTH'], categories=month_order, ordered=True)

# Filter crops where ALL production values are < 200
crop_max = avg_df.groupby('CROP')['Production(mt)'].max()
low_producing_crops = crop_max[crop_max < 200].index
filtered_df = avg_df[avg_df['CROP'].isin(low_producing_crops)].copy()

# Set 'MONTH' as ordered categorical for filtered_df too
filtered_df['MONTH'] = pd.Categorical(filtered_df['MONTH'], categories=month_order, ordered=True)

# Create subplots
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# First subplot - full data
sns.lineplot(
    data=avg_df,
    x='MONTH',
    y='Production(mt)',
    hue='CROP',
    errorbar=None,
    linewidth=1,
    ax=axes[0]
)
axes[0].set_title('Monthly Production by Crop Type (All Data)')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Production (mt)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend(title='Crops', fontsize=7, title_fontsize=10)

# Second subplot - filtered crops
sns.lineplot(
    data=filtered_df,
    x='MONTH',
    y='Production(mt)',
    hue='CROP',
    errorbar=None,
    linewidth=1,
    ax=axes[1]
)
axes[1].set_title('Monthly Production (Crops with All Values < 200 mt)')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Production (mt)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend(title='Crops', fontsize=7, title_fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
"""
The bar chart illustrates the production levels (in metric tons) of various crops
across two different farm types: irrigated and rainfed. Each group on the x-axis
represents a farm type, with colored bars indicating the production levels of
individual crops within each category.

The chart reveals that cabbage and white potato are the most highly produced crops,
followed by Chinese cabbage and carrots. Additionally, overall production is
notably higher on rainfed farms compared to irrigated farms.
"""


plt.figure(figsize=(14, 8))
ax = sns.barplot(data=df2, x='FARM TYPE', y='Production(mt)', hue='CROP', errorbar=None)

for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.text(
            p.get_x() + p.get_width() / 2,
            height * 0.5,
            f'{height:,.0f}',
            ha='center', va='center',
            color='white', fontsize=9, fontweight='bold'
        )

plt.legend(title='CROPS')
plt.tight_layout()
plt.show()


In [ ]:
"""
The bar chart illustrates the crop production levels (in metric tons) of various municipalities
across both irrigated and rainfed farm types.

The data show that the municipalities of Atok, Bakun, Buguias, Kabayan, Kibungan, and Mankayan
consistently produce significantly higher crop volumes compared to other municipalities.
While rainfed farms generally yield more than irrigated ones, Buguias stands out as an exception,
with higher production under irrigated conditions.

These high-performing municipalities achieve average production levels reaching up to
1,000 metric tons, whereas other municipalities typically produce only up to 100 metric tons.
"""


plt.figure(figsize=(14, 8))
ax = sns.barplot(data=df2, x='MUNICIPALITY', y='Production(mt)', hue='FARM TYPE', errorbar=None)

for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.text(
            p.get_x() + p.get_width() / 2,
            height * 0.5,
            f'{height:,.0f}',
            ha='center', va='center',
            color='white', fontsize=9, fontweight='bold'
        )

plt.legend(title='CROPS')
plt.tight_layout()
plt.show()


In [ ]:
"""
This figure presents monthly crop production trends for each year in the dataset using individual line plots.

Each subplot represents a single year, showing how production varies across months.
The x-axis displays the months of the year, highlighting seasonal patterns.
The y-axis indicates production volume in metric tons, standardized across all subplots for consistent comparison.
This layout enables clear visual comparison of harvest volumes and seasonal trends across different years.

The plots reveal that the majority of harvesting occurs between May and August, with additional activity observed in December and January.
In contrast, the period from February to April consistently shows the lowest production levels.
"""



years = df2['YEAR'].unique()
n = len(years)
cols = 3
rows = math.ceil(n / cols)

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4), sharey=True)
axes = axes.flatten()

# Plot each year
for i, year in enumerate(years):
    data = df2[df2['YEAR'] == year]
    sns.lineplot(ax=axes[i], data=data, x='MONTH', y='Production(mt)', marker='o', color='royalblue', errorbar=None)
    axes[i].set_title(year, fontsize=10)
    axes[i].set_xlabel("MONTH", size=10)
    axes[i].tick_params(axis='x', labelrotation=45, labelsize=7)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Monthly Harvest Trend Across Years', fontsize=16)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
#import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
#import numpy as np

# Clean the data: drop NaN or infinite values in the production column
df2_clean = df2.copy()
df2_clean = df2_clean.replace([np.inf, -np.inf], np.nan)
df2_clean = df2_clean.dropna(subset=['Production(mt)'])

# Get list of unique years
unique_years = df2_clean['YEAR'].dropna().unique()

# Set seaborn style
sns.set(style="whitegrid")

# Create one plot per year
for year in unique_years:
    plt.figure(figsize=(10, 6))
    subset = df2_clean[df2_clean['YEAR'] == year]

    # Group by crop and calculate average production, then sort
    avg_prod = (
        subset.groupby('CROP')['Production(mt)']
        .mean()
        .reset_index()
        .sort_values('Production(mt)', ascending=False)
    )

    # Create barplot and store Axes
    ax = sns.barplot(
        data=avg_prod,
        x='CROP',
        y='Production(mt)',
        hue='CROP',
        palette='viridis',
        legend=False,
        order=avg_prod['CROP']
    )

    # Add data labels on top of each bar
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f', label_type='edge', padding=3, fontsize=10)

    # Plot formatting
    plt.title(f'Average Crop Production in {int(year)}')
    plt.xlabel('Crops', fontsize=10)
    plt.ylabel('Average Production (mt)')
    plt.xticks(rotation=45, fontsize=9)
    plt.tight_layout(pad=4.0)
    plt.show()


 # START
# 1. Yield Prediction Dashboard
# Visualize predicted crop yields per season/year
# Compare actual vs. predicted yields
# Include confidence scores from ML models

In [ ]:
# Use Random Forest as primary model for confidence scores

print("\nData types of X_test before prediction:")
print(X_test.dtypes)

# Get predictions using the full pipeline
rf_predictions = rf_model.predict(X_test)

# Get individual tree predictions for confidence interval
individual_tree_predictions = np.array([
    tree.predict(rf_model.best_estimator_.named_steps['preprocessor'].transform(X_test))
    for tree in rf_model.best_estimator_.named_steps['model'].estimators_
])

# Calculate the standard deviation of the tree predictions for each data point
std_dev_predictions = np.std(individual_tree_predictions, axis=0)

# Use 1.96 multiplier for 95% confidence interval
confidence_interval = 1.96 * std_dev_predictions

# Plot Actual vs. Predicted for Random Forest with confidence intervals
plt.figure(figsize=(10, 6))
plt.errorbar(y_test, rf_predictions, yerr=confidence_interval, fmt='o', alpha=0.6, label='RF Predictions (95% CI)')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.xlabel('Actual Production (mt)')
plt.ylabel('Predicted Production (mt)')
plt.title('Random Forest: Actual vs Predicted Production with 95% Confidence Intervals')
plt.legend()
plt.show()

# Compute evaluation metrics
r2 = r2_score(y_test, rf_predictions)
mae = mean_absolute_error(y_test, rf_predictions)
mse = mean_squared_error(y_test, rf_predictions)
rmse = np.sqrt(mse)

print(f"\nRandom Forest Evaluation Metrics:")
print(f"R² Score: {r2:.3f}")
print(f"Mean Absolute Error: {mae:.3f}")
print(f"Mean Squared Error: {mse:.3f}")
print(f"Root Mean Squared Error: {rmse:.3f}")

# Scatter plot (Actual vs Predicted)
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=rf_predictions, alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.xlabel('Actual Production (mt)')
plt.ylabel('Predicted Production (mt)')
plt.title('Random Forest: Actual vs Predicted')
plt.show()


In [ ]:
rf_model
rf_predictions = rf_model.predict(X_test)

visualization_df = pd.DataFrame({
    'Actual Production': y_test.values,
    'Predicted Production': rf_predictions,
    'MONTH': df2.loc[test_index, 'MONTH'].values
})

monthly_production = visualization_df.groupby('MONTH').agg(
    mean_actual=('Actual Production', 'mean'),
    mean_predicted=('Predicted Production', 'mean'),
    std_predicted=('Predicted Production', 'std')
).reset_index()

month_order = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
               'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

monthly_production['MONTH'] = pd.Categorical(monthly_production['MONTH'],
                                             categories=month_order, ordered=True)
monthly_production = monthly_production.sort_values('MONTH')

monthly_counts = visualization_df.groupby('MONTH').size().reindex(month_order).fillna(0).values
monthly_production['se_predicted'] = monthly_production['std_predicted'] / np.sqrt(monthly_counts)
monthly_production['ci_lower'] = monthly_production['mean_predicted'] - 1.96 * monthly_production['se_predicted']
monthly_production['ci_upper'] = monthly_production['mean_predicted'] + 1.96 * monthly_production['se_predicted']

plt.figure(figsize=(12, 6))
plt.plot(monthly_production['MONTH'], monthly_production['mean_actual'], marker='o', linestyle='-', label='Actual Production')
plt.plot(monthly_production['MONTH'], monthly_production['mean_predicted'], marker='o', linestyle='-', color='red', label='Predicted Production')
plt.fill_between(monthly_production['MONTH'], monthly_production['ci_lower'], monthly_production['ci_upper'], color='red', alpha=0.2, label='95% Confidence Interval')
plt.xlabel('Month')
plt.ylabel('Average Production (mt)')
plt.title('Average Actual vs. Predicted Production by Month with 95% Confidence Interval')
plt.legend()
plt.grid(True)
plt.show()

visualization_df = pd.DataFrame({
    'Actual Production': y_test.values,
    'Predicted Production': rf_predictions,
    'YEAR': df2.loc[test_index, 'YEAR'].values
})

yearly_production = visualization_df.groupby('YEAR').agg(
    mean_actual=('Actual Production', 'mean'),
    mean_predicted=('Predicted Production', 'mean'),
    std_predicted=('Predicted Production', 'std')
).reset_index()

yearly_counts = visualization_df.groupby('YEAR').size().values
yearly_production['se_predicted'] = yearly_production['std_predicted'] / np.sqrt(yearly_counts)
yearly_production['ci_lower'] = yearly_production['mean_predicted'] - 1.96 * yearly_production['se_predicted']
yearly_production['ci_upper'] = yearly_production['mean_predicted'] + 1.96 * yearly_production['se_predicted']

plt.figure(figsize=(12, 6))
plt.plot(yearly_production['YEAR'], yearly_production['mean_actual'], marker='o', linestyle='-', label='Actual Production')
plt.plot(yearly_production['YEAR'], yearly_production['mean_predicted'], marker='o', linestyle='-', color='red', label='Predicted Production')
plt.fill_between(yearly_production['YEAR'], yearly_production['ci_lower'], yearly_production['ci_upper'], color='red', alpha=0.2, label='95% Confidence Interval')
plt.xlabel('Year')
plt.ylabel('Average Production (mt)')
plt.title('Average Actual vs. Predicted Production by Year with 95% Confidence Interval')
plt.legend()
plt.grid(True)
plt.xticks(yearly_production['YEAR'], rotation=45, ha='right')
plt.show()


# 2. Trend Analysis:
# Show historical trends in crop production, price, and area planted
# Use line charts, bar graphs, and heatmaps

In [ ]:
# Plotting historical trends for Production and Area Planted

# Use df2 for plotting as it contains the necessary data
if df2 is not None:
    # Production trend
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df2, x='YEAR', y='Production(mt)', marker='o', color='darkgreen', errorbar=None)
    plt.title('Yearly Trend in Crop Production')
    plt.xlabel('Year')
    plt.ylabel('Production (mt)')
    plt.grid(True)
    plt.show()

    # Area Planted trend (assuming 'Area planted(ha)' is available in df2)
    if 'Area planted(ha)' in df2.columns:
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=df2, x='YEAR', y='Area planted(ha)', marker='o', color='orange', errorbar=None)
        plt.title('Yearly Trend in Area Planted')
        plt.xlabel('Year')
        plt.ylabel('Area Planted (ha)')
        plt.grid(True)
        plt.show()
    else:
        print("'Area planted(ha)' column not found in the available dataframe. Skipping this plot.")


    # Price trend (assuming 'Price(Php/kg)' is available in df2)
    if 'Price(Php/kg)' in df2.columns:
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=df2, x='YEAR', y='Price(Php/kg)', marker='o', color='purple', errorbar=None)
        plt.title('Yearly Trend in Crop Price')
        plt.xlabel('Year')
        plt.ylabel('Price (Php/kg)')
        plt.grid(True)
        plt.show()
    else:
        print("'Price(Php/kg)' column not found in the available dataframe. Skipping this plot.")

# 3. Crop Performance Comparison:
# Compare different crops based on yield, profitability, and maturity duration
# Highlight top-performing crops per season or location

In [ ]:
municipality_crop_performance = df2.groupby(['MUNICIPALITY', 'CROP']).agg(
    mean_production=('Production(mt)', 'mean'),
    mean_productivity=('Productivity(mt/ha)', 'mean')
).reset_index()
display(municipality_crop_performance)

In [ ]:
monthly_crop_performance = df2.groupby(['MONTH', 'CROP']).agg(
    mean_production=('Production(mt)', 'mean'),
    mean_productivity=('Productivity(mt/ha)', 'mean')
).reset_index()

In [ ]:
# Define the order of months for plotting
month_order = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# Convert 'MONTH' to ordered categorical for sorting on x-axis
monthly_crop_performance['MONTH'] = pd.Categorical(monthly_crop_performance['MONTH'], categories=month_order, ordered=True)

# Sort by month
monthly_crop_performance = monthly_crop_performance.sort_values('MONTH')

# Create line plot for mean production by crop
plt.figure(figsize=(14, 7))
sns.lineplot(
    data=monthly_crop_performance,
    x='MONTH',
    y='mean_production',
    hue='CROP',
    errorbar=None,
    linewidth=1
)
plt.title('Average Crop Production by Month')
plt.xlabel('Month')
plt.ylabel('Average Production (mt)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Crops', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Create line plot for mean productivity by crop
plt.figure(figsize=(14, 7))
sns.lineplot(
    data=monthly_crop_performance,
    x='MONTH',
    y='mean_productivity',
    hue='CROP',
    errorbar=None,
    linewidth=1
)
plt.title('Average Crop Productivity by Month')
plt.xlabel('Month')
plt.ylabel('Average Productivity (mt/ha)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Crops', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate average production and productivity per crop
crop_performance = df2.groupby('CROP').agg(
    mean_production=('Production(mt)', 'mean'),
    mean_productivity=('Productivity(mt/ha)', 'mean')
).reset_index()

# Sort by mean production for plotting
crop_performance_production = crop_performance.sort_values('mean_production', ascending=False)

# Create bar plot for average production
plt.figure(figsize=(12, 6))
sns.barplot(data=crop_performance_production, x='CROP', y='mean_production', palette='viridis')
plt.title('Average Crop Production (mt)')
plt.xlabel('Crop')
plt.ylabel('Average Production (mt)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Sort by mean productivity for plotting
crop_performance_productivity = crop_performance.sort_values('mean_productivity', ascending=False)

# Create bar plot for average productivity
plt.figure(figsize=(12, 6))
sns.barplot(data=crop_performance_productivity, x='CROP', y='mean_productivity', palette='viridis')
plt.title('Average Crop Productivity (mt/ha)')
plt.xlabel('Crop')
plt.ylabel('Average Productivity (mt/ha)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
monthly_crop_performance = df2.groupby(['MONTH', 'CROP']).agg(
    mean_production=('Production(mt)', 'mean'),
    mean_productivity=('Productivity(mt/ha)', 'mean')
).reset_index()

In [ ]:
# Define the order of months for plotting
month_order = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# Convert 'MONTH' to ordered categorical for sorting on x-axis
monthly_crop_performance['MONTH'] = pd.Categorical(monthly_crop_performance['MONTH'], categories=month_order, ordered=True)

# Sort by month
monthly_crop_performance = monthly_crop_performance.sort_values('MONTH')

# Create line plot for mean production by crop
plt.figure(figsize=(14, 7))
sns.lineplot(
    data=monthly_crop_performance,
    x='MONTH',
    y='mean_production',
    hue='CROP',
    errorbar=None,
    linewidth=1
)
plt.title('Average Crop Production by Month')
plt.xlabel('Month')
plt.ylabel('Average Production (mt)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Crops', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Create line plot for mean productivity by crop
plt.figure(figsize=(14, 7))
sns.lineplot(
    data=monthly_crop_performance,
    x='MONTH',
    y='mean_productivity',
    hue='CROP',
    errorbar=None,
    linewidth=1
)
plt.title('Average Crop Productivity by Month')
plt.xlabel('Month')
plt.ylabel('Average Productivity (mt/ha)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Crops', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
municipality_crop_performance = df2.groupby(['MUNICIPALITY', 'CROP']).agg(
    mean_production=('Production(mt)', 'mean'),
    mean_productivity=('Productivity(mt/ha)', 'mean')
).reset_index()
display(municipality_crop_performance)

In [ ]:
# Sort by mean production for plotting
municipality_crop_performance_production = municipality_crop_performance.sort_values('mean_production', ascending=False)

# Create bar plot for average production
plt.figure(figsize=(14, 7))
sns.barplot(data=municipality_crop_performance_production, x='MUNICIPALITY', y='mean_production', hue='CROP', palette='viridis')
plt.title('Average Crop Production (mt) by Municipality')
plt.xlabel('Municipality')
plt.ylabel('Average Production (mt)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Crops', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Sort by mean productivity for plotting
municipality_crop_performance_productivity = municipality_crop_performance.sort_values('mean_productivity', ascending=False)

# Create bar plot for average productivity
plt.figure(figsize=(14, 7))
sns.barplot(data=municipality_crop_performance_productivity, x='MUNICIPALITY', y='mean_productivity', hue='CROP', palette='viridis')
plt.title('Average Crop Productivity (mt/ha) by Municipality')
plt.xlabel('Municipality')
plt.ylabel('Average Productivity (mt/ha)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Crops', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 4. Seasonal Forecasting
#Use time series models to forecast future crop trends

In [ ]:
# Prepare data for time series forecasting
# Aggregate production data by year
yearly_production_ts = df2.groupby('YEAR').agg(
    Production=('Production(mt)', 'mean')
).reset_index()

# Set YEAR as index for time series analysis
yearly_production_ts['YEAR'] = pd.to_datetime(yearly_production_ts['YEAR'], format='%Y')
yearly_production_ts = yearly_production_ts.set_index('YEAR')

# Create lagged features for time series forecasting
def create_lagged_features(df, lag=1):
    df_lagged = df.copy()
    for i in range(1, lag + 1):
        df_lagged[f'Production_lag_{i}'] = df_lagged['Production'].shift(i)
    df_lagged = df_lagged.dropna()
    return df_lagged

lag_order = 1 # Using a lag of 1 year
yearly_production_ts_lagged = create_lagged_features(yearly_production_ts, lag_order)

# Define features (X) and target (y) for time series model
X_ts = yearly_production_ts_lagged.drop('Production', axis=1)
y_ts = yearly_production_ts_lagged['Production']

# Split data for time series training and testing (using the last year for testing)
X_train_ts = X_ts.iloc[:-1]
X_test_ts = X_ts.iloc[-1:]
y_train_ts = y_ts.iloc[:-1]
y_test_ts = y_ts.iloc[-1:]


# Train Time Series models
ts_pipelines = {
    'Random Forest TS': make_pipeline(StandardScaler(), RandomForestRegressor())
}

ts_grid = {
    'Random Forest TS':{
        'randomforestregressor__max_depth': [None, 100, 200]
    }

}


fitted_ts_models = {}
for algo, pipeline in ts_pipelines.items():
    print(f'Training the {algo} model')
    model = GridSearchCV(pipeline, ts_grid[algo], n_jobs =-1, cv=3) # Using lower CV for small dataset
    model.fit(X_train_ts, y_train_ts)
    fitted_ts_models[algo] = model
    print(model.best_params_)
print('Done')

In [ ]:
# Access the Random Forest time series model
rf_ts_model = fitted_ts_models['Random Forest TS']

# Forecast future crop trends (next 5 years)
forecast_periods = 5

# Use the last known lagged data from the training set for the initial forecast
last_year_data = X_train_ts.iloc[-1].values.tolist()
forecasted_values = []

for _ in range(forecast_periods):
    # Predict the next year's production using Random Forest
    next_prediction = rf_ts_model.predict(np.array(last_year_data).reshape(1, -1))[0]
    forecasted_values.append(next_prediction)

    # Update lagged features (shift window forward)
    last_year_data = [next_prediction] + last_year_data[:-1]

# Create a date index for the forecasted values
last_year = yearly_production_ts_lagged.index[-1]
forecast_years = pd.date_range(start=last_year + pd.DateOffset(years=1), periods=forecast_periods, freq='YS')
forecast_series = pd.Series(forecasted_values, index=forecast_years)

# Show the forecasted results
print("Future Forecasts (Random Forest TS):")
print(forecast_series)

# Combine historical data and forecast for plotting
historical_data = yearly_production_ts['Production']

plt.figure(figsize=(12, 6))
plt.plot(historical_data.index, historical_data, marker='o', linestyle='-', label='Historical Production')
plt.plot(forecast_series.index, forecast_series, marker='o', linestyle='--', color='red', label='Forecasted Production (Random Forest TS)')

plt.xlabel('Year')
plt.ylabel('Average Production (mt)')
plt.title('Crop Production Forecast using Random Forest Time Series')
plt.legend()
plt.grid(True)

# Set x-axis ticks to show all years from 2015 to the end of forecast
all_years = pd.date_range(start=historical_data.index[0], end=forecast_series.index[-1], freq='YS')
plt.xticks(all_years, [year.year for year in all_years], rotation=45)

plt.show()

# REVISION: Work with Christian on the prediction of each crop

In [ ]:
def create_lagged_features(series, n_lags=12):
    df = pd.DataFrame(series).copy()
    df.columns = ['y']
    for i in range(1, n_lags + 1):
        df[f'lag_{i}'] = df['y'].shift(i)
    df = df.dropna()
    return df

# FORECASTED CROPS PER MUNICIPALITY: PRODUCTION

In [ ]:
# Map month names to numbers
month_map = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}
df['MONTH_NUM'] = df['MONTH'].map(month_map) # Use df here to create the column


# Use the original df to create the 'DATE' column and monthly data
df['DATE'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH_NUM'].astype(int).astype(str) + '-01')


# Group by municipality + crop + year-month
monthly_crop_data = df.groupby(['MUNICIPALITY', 'CROP', 'DATE'])['Production(mt)'].sum().reset_index()

# Convert 'Production(mt)' to numeric and drop rows with non-numeric values
monthly_crop_data['Production(mt)'] = pd.to_numeric(monthly_crop_data['Production(mt)'], errors='coerce')
monthly_crop_data.dropna(subset=['Production(mt)'], inplace=True)


def create_lagged_features(series, n_lags=12):
    df = pd.DataFrame(series).copy()
    df.columns = ['y']
    for i in range(1, n_lags + 1):
        df[f'lag_{i}'] = df['y'].shift(i)
    df = df.dropna()
    return df


def forecast_crop_rf(municipality, crop_name, monthly_crop_data, forecast_months=60, n_lags=12):
    print(f"--- Processing {municipality} - {crop_name} ---")
    crop_data = monthly_crop_data[
        (monthly_crop_data['MUNICIPALITY'] == municipality) &
        (monthly_crop_data['CROP'] == crop_name)
    ].sort_values('DATE').set_index('DATE')['Production(mt)']

    if crop_data.empty or len(crop_data) < n_lags + 1:
        print(f"Not enough data for {municipality} - {crop_name}")
        return None, None

    lagged = create_lagged_features(crop_data, n_lags=n_lags)
    X = lagged.drop(columns=['y'])
    y = lagged['y']

    # print(f"Data types of X before fitting: {X.dtypes}")
    # print(f"Data types of y before fitting: {y.dtypes}")
    # print(f"X head:\n{X.head()}")
    # print(f"y head:\n{y.head()}")


    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X, y)

    last_known = lagged.iloc[-1, 1:].values
    preds = []
    current_date = crop_data.index[-1]

    # print(f"Initial last_known: {last_known}, dtype: {last_known.dtype}")

    for i in range(forecast_months):
        try:
            # Ensure last_known is numeric before prediction
            last_known_numeric = last_known.astype(float)
            # match feature names
            feature_names = [f'lag_{i}' for i in range(1, n_lags + 1)]
            next_pred = model.predict(pd.DataFrame([last_known_numeric], columns=feature_names))[0]
            preds.append(next_pred)

            # Update last_known with the new prediction (ensure it's numeric)
            last_known = np.roll(last_known, 1)
            last_known[0] = float(next_pred) # Ensure assignment is float

            # print(f"Iteration {i}: next_pred = {next_pred}, updated last_known = {last_known}")

        except ValueError as e:
            print(f"ValueError in forecasting loop at iteration {i}: {e}")
            # print(f"last_known at error: {last_known}, dtype: {last_known.dtype}")
            break # Exit the loop on error


    forecast_index = pd.date_range(start=current_date + pd.DateOffset(months=1), periods=forecast_months, freq='MS')
    forecast_series = pd.Series(preds, index=forecast_index)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(crop_data.index, crop_data.values, marker='o', label="Historical")
    ax.plot(forecast_series.index, forecast_series.values, marker='o', linestyle='--', color='red', label="Forecast")
    ax.set_title(f"Forecasted Production(mt): {crop_name} in {municipality}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Production (mt)")
    ax.legend()
    ax.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()

    return fig, ax

# --- Generate forecasts for all pairs ---
figures = []
municipalities = monthly_crop_data['MUNICIPALITY'].unique()
crops = monthly_crop_data['CROP'].unique()

for m in municipalities:
    for c in crops:
        fig, ax = forecast_crop_rf(m, c, monthly_crop_data, forecast_months=60, n_lags=12)
        if fig is not None:
            figures.append(fig)

print(f"Generated {len(figures)} plots.")
for fig in figures:
    plt.show(fig)

# FORECASTED CROPS PER MUNICIPALITY: PRODUCTIVITY

In [ ]:
df['MONTH_NUM'] = df['MONTH'].map(month_map)

# Create a DATE column for grouping (first day of each month)
df['DATE'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH_NUM'].astype(str) + '-01')

# --- 2. Group by Municipality + Crop + Date ---
# We'll aggregate productivity (mean in case of multiple records)
monthly_productivity_data = (
    df.groupby(['MUNICIPALITY', 'CROP', 'DATE'])['Productivity(mt/ha)']
    .mean()
    .reset_index()
)

# Convert Productivity to numeric (safety)
monthly_productivity_data['Productivity(mt/ha)'] = pd.to_numeric(
    monthly_productivity_data['Productivity(mt/ha)'], errors='coerce'
)
monthly_productivity_data.dropna(subset=['Productivity(mt/ha)'], inplace=True)

# --- 3. Helper function to create lagged features ---
def create_lagged_features(series, n_lags=12):
    df_temp = pd.DataFrame(series).copy()
    df_temp.columns = ['y']
    for i in range(1, n_lags + 1):
        df_temp[f'lag_{i}'] = df_temp['y'].shift(i)
    df_temp = df_temp.dropna()
    return df_temp

# --- 4. Forecast function using Random Forest ---
def forecast_productivity_rf(municipality, crop_name, monthly_data, forecast_months=60, n_lags=12):
    print(f"--- Processing {municipality} - {crop_name} ---")

    crop_data = monthly_data[
        (monthly_data['MUNICIPALITY'] == municipality) &
        (monthly_data['CROP'] == crop_name)
    ].sort_values('DATE').set_index('DATE')['Productivity(mt/ha)']

    if crop_data.empty or len(crop_data) < n_lags + 1:
        print(f"Not enough data for {municipality} - {crop_name}")
        return None, None

    lagged = create_lagged_features(crop_data, n_lags)
    X = lagged.drop(columns=['y'])
    y = lagged['y']

    model = RandomForestRegressor(n_estimators=200, random_state=42)
    model.fit(X, y)

    last_known = lagged.iloc[-1, 1:].values
    preds = []
    current_date = crop_data.index[-1]

    for i in range(forecast_months):
        try:
            last_known_numeric = last_known.astype(float)
            feature_names = [f'lag_{i}' for i in range(1, n_lags + 1)]
            next_pred = model.predict(pd.DataFrame([last_known_numeric], columns=feature_names))[0]
            preds.append(next_pred)

            last_known = np.roll(last_known, 1)
            last_known[0] = float(next_pred)
        except ValueError as e:
            print(f"ValueError at iteration {i}: {e}")
            break

    forecast_index = pd.date_range(start=current_date + pd.DateOffset(months=1), periods=forecast_months, freq='MS')
    forecast_series = pd.Series(preds, index=forecast_index)

    # --- Visualization ---
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(crop_data.index, crop_data.values, marker='o', label="Historical")
    ax.plot(forecast_series.index, forecast_series.values, marker='o', linestyle='--', color='red', label="Forecast")
    ax.set_title(f"Forecasted Productivity: {crop_name} in {municipality}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Productivity (mt/ha)")
    ax.legend()
    ax.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()

    return fig, ax

# --- 5. Generate forecasts for all crop-municipality pairs ---
figures = []
municipalities = monthly_productivity_data['MUNICIPALITY'].unique()
crops = monthly_productivity_data['CROP'].unique()

for m in municipalities:
    for c in crops:
        fig, ax = forecast_productivity_rf(m, c, monthly_productivity_data, forecast_months=60, n_lags=12)
        if fig is not None:
            figures.append(fig)

print(f"✅ Generated {len(figures)} productivity forecast plots.")

for fig in figures:
    plt.show(fig)

In [ ]:
# Map month names to numbers
month_map = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}
df2['MONTH_NUM'] = df2['MONTH'].map(month_map) # Use df2 here

# Drop rows where MONTH_NUM is NaN to handle invalid month names
df2_cleaned = df2.dropna(subset=['MONTH_NUM']).copy()

# Create a 'Date' column for chronological ordering and plotting
df2_cleaned['Date'] = pd.to_datetime(df2_cleaned['YEAR'].astype(str) + '-' + df2_cleaned['MONTH_NUM'].astype(int).astype(str) + '-01')


# Filter data for the required time frame (2015-2024)
df_filtered = df2_cleaned[(df2_cleaned['YEAR'] >= 2015) & (df2_cleaned['YEAR'] <= 2024)].copy() # Use df2 here and create a copy

# Group by Date and CROP to get the mean monthly productivity for each crop
df_monthly_productivity = df_filtered.groupby(['Date', 'CROP'])['Productivity(mt/ha)'].mean().reset_index()

# --- Debugging Prints ---
print("df_monthly_productivity head:")
display(df_monthly_productivity.head())
print("\nUnique crops in df_monthly_productivity:")
print(df_monthly_productivity['CROP'].unique())
# --- End Debugging Prints ---


# --- Plotting ---

plt.figure(figsize=(15, 8))

# Get unique crops
crops = df_monthly_productivity['CROP'].unique()

# Plot the productivity trend for each crop
for crop in crops:
    crop_data = df_monthly_productivity[df_monthly_productivity['CROP'] == crop]
    plt.plot(crop_data['Date'], crop_data['Productivity(mt/ha)'], label=crop, marker='.', linestyle='-')

# Format the plot
plt.title('Monthly Average Productivity (mt/ha) by Crop (2015 - 2024)')
plt.xlabel('Date')
plt.ylabel('Average Productivity (mt/ha)')
plt.grid(True, linestyle='--', alpha=0.6)

# Set x-axis major ticks to years and format
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45, ha='right')

# Place the legend outside the plot area for clarity
plt.legend(title='Crop', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 0.85, 1])

# Show the plot
plt.show()

In [ ]:
# Convert numeric columns and drop rows with missing values
df['Area planted(ha)'] = pd.to_numeric(df['Area planted(ha)'], errors='coerce')
df['Area harvested(ha)'] = pd.to_numeric(df['Area harvested(ha)'], errors='coerce')
df['Productivity(mt/ha)'] = pd.to_numeric(df['Productivity(mt/ha)'], errors='coerce')
df = df.dropna(subset=['Area planted(ha)', 'Area harvested(ha)', 'Productivity(mt/ha)'])

# Define features and target
features = ['YEAR', 'MONTH_NUM', 'Area planted(ha)', 'Area harvested(ha)', 'MUNICIPALITY', 'FARM TYPE', 'CROP']
target = 'Productivity(mt/ha)'

X = df[features]
y = df[target]

# One-hot encode categorical features
categorical_features = ['MUNICIPALITY', 'FARM TYPE', 'CROP']
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)
model_features = X_encoded.columns.tolist()

# Train Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_encoded, y)

# --- Forecasting (2025-2030) ---
start_year = 2025
end_year = 2030

# Get unique municipality-farm-crop combinations and historical averages for numeric features
unique_combinations = df[['MUNICIPALITY', 'FARM TYPE', 'CROP']].drop_duplicates().reset_index(drop=True)
avg_features = df.groupby(['MUNICIPALITY', 'FARM TYPE', 'CROP'])[['Area planted(ha)', 'Area harvested(ha)']].mean().reset_index()

forecast_data_list = []
for year in range(start_year, end_year + 1):
    for month_num in range(1, 13):
        current_date = pd.to_datetime(f'{year}-{month_num:02d}-01')
        for _, combo in unique_combinations.iterrows():
            avg_vals = avg_features[
                (avg_features['MUNICIPALITY'] == combo['MUNICIPALITY']) &
                (avg_features['FARM TYPE'] == combo['FARM TYPE']) &
                (avg_features['CROP'] == combo['CROP'])
            ].iloc[0]

            row = {
                'YEAR': year,
                'MONTH_NUM': month_num,
                'Date': current_date,
                'Area planted(ha)': avg_vals['Area planted(ha)'],
                'Area harvested(ha)': avg_vals['Area harvested(ha)'],
                'MUNICIPALITY': combo['MUNICIPALITY'],
                'FARM TYPE': combo['FARM TYPE'],
                'CROP': combo['CROP']
            }
            forecast_data_list.append(row)

df_forecast = pd.DataFrame(forecast_data_list)

# Prepare features for prediction
X_forecast = df_forecast[features]
X_forecast_encoded = pd.get_dummies(X_forecast, columns=categorical_features, drop_first=True)

# Align columns with training features
missing_cols = set(model_features) - set(X_forecast_encoded.columns)
for c in missing_cols:
    X_forecast_encoded[c] = 0
X_forecast_encoded = X_forecast_encoded[model_features]

# Predict productivity
df_forecast['Predicted Productivity (mt/ha)'] = rf_model.predict(X_forecast_encoded)

# --- Plot forecast by crop ---
df_forecast_monthly_crop = df_forecast.groupby(['Date', 'CROP'])['Predicted Productivity (mt/ha)'].mean().reset_index()

plt.figure(figsize=(15, 8))
for crop in df_forecast_monthly_crop['CROP'].unique():
    crop_data = df_forecast_monthly_crop[df_forecast_monthly_crop['CROP'] == crop]
    plt.plot(crop_data['Date'], crop_data['Predicted Productivity (mt/ha)'], label=crop)

plt.title('Forecasted Monthly Average Productivity (mt/ha) by Crop (2025 - 2030)')
plt.xlabel('Date')
plt.ylabel('Average Predicted Productivity (mt/ha)')
plt.grid(True, linestyle='--', alpha=0.6)

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator(1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45, ha='right')

plt.legend(title='Crop', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.show()

In [ ]:
df['Production(mt)'] = pd.to_numeric(df['Production(mt)'], errors='coerce')
df_clean = df.dropna(subset=['Production(mt)'])

# --- Get available years ---
available_years = sorted(df_clean['YEAR'].unique())
print("Available years:", available_years)

# --- Ask user until a valid year is entered ---
while True:
    try:
        year = int(input("\nEnter the year to view top 5 crops: "))
        if year not in available_years:
            print(f"Year {year} not found in dataset. Please try again.")
            continue
        break
    except ValueError:
        print("Invalid input. Please enter a numeric year.")

# --- Filter for chosen year ---
year_data = df_clean[df_clean['YEAR'] == year]

# --- Compute top 5 crops ---
top5 = (year_data.groupby('CROP')['Production(mt)']
        .sum()
        .reset_index()
        .sort_values(by='Production(mt)', ascending=False)
        .head(5))

# --- Plot vertical bar chart (highest first) ---
# NOTE: Standard sns.barplot does NOT add values to the bars, fixing the previous issue.
plt.figure(figsize=(8, 5))
sns.barplot(
    data=top5,
    x='CROP',
    y='Production(mt)',
    palette='YlGn',
    order=top5['CROP']  # ensures bars are in descending order
)
plt.title(f"Top 5 Crops by Production in {year}", fontsize=14)
plt.xlabel("Crop")
plt.ylabel("Total Production (mt)")
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

# --- Text-based summary ---
print(f"\nTop 5 Crops in {year} (by total production):\n")
for i, row in enumerate(top5.itertuples(index=False), start=1):
    production_value = getattr(row, 'Production(mt)')
    # Use consistent formatting for cleaner output
    print(f"{i}. {row.CROP}: {production_value:,.2f} metric tons")

# --- Insight ---
# Access the top row data directly for concise code
top_crop_name = top5.iloc[0]['CROP']
top_crop_production = top5.iloc[0]['Production(mt)']
print(f"\nInsight: The highest-producing crop in {year} was "
      f"{top_crop_name} with {top_crop_production:,.2f} metric tons.")

In [ ]:
df['Production(mt)'] = pd.to_numeric(df['Production(mt)'], errors='coerce')
df_clean = df.dropna(subset=['Production(mt)'])

# --- Get available crops (using UP-CASED values for matching) ---
available_crops = sorted(df_clean['CROP'].str.upper().unique())
print("Available crops:", sorted(df_clean['CROP'].unique()))

# --- Ask user until a valid crop is entered (case-insensitive) ---
while True:
    # Read input and convert to UPPERCASE immediately
    crop_input = input("\nEnter the crop to view top 5 municipalities: ").strip().upper()

    if crop_input not in available_crops:
        print(f"Crop '{crop_input}' not found in dataset. Please try again.")
        continue
    break

# --- Filter for chosen crop (using case-insensitive comparison) ---
# Filter the original CROP column by converting it to uppercase first.
crop_data = df_clean[df_clean['CROP'].str.upper() == crop_input]

# --- Compute top 5 municipalities ---
top5_municipalities = (crop_data.groupby('MUNICIPALITY')['Production(mt)']
                        .sum()
                        .reset_index()
                        .sort_values(by='Production(mt)', ascending=False)
                        .head(5))

# --- Plot setup (bars only, y-axis accurate, no values on bars) ---
plt.figure(figsize=(8, 5))
sns.barplot(
    data=top5_municipalities,
    x='MUNICIPALITY',
    y='Production(mt)',
    palette='YlGn',
    order=top5_municipalities['MUNICIPALITY']  # ensures correct order
)

plt.title(f"Top 5 Municipalities Producing {crop_input}", fontsize=14)
plt.xlabel("Municipality")
plt.ylabel("Total Production (mt)")
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

# --- Text-based summary ---
print(f"\nTop 5 Municipalities Producing {crop_input} (by total production):\n")
for i, row in enumerate(top5_municipalities.itertuples(index=False), start=1):
    production_value = getattr(row, 'Production(mt)')
    print(f"{i}. {row.MUNICIPALITY}: {production_value:,.2f} metric tons")

# --- Insight ---
top_municipality = top5_municipalities.iloc[0]
top_municipality_name = top_municipality['MUNICIPALITY']
top_municipality_production = top_municipality['Production(mt)']
print(f"\nInsight: The municipality with the highest production of {crop_input} is "
      f"{top_municipality_name} with {top_municipality_production:,.2f} metric tons.")